In [1]:
import numpy as np
from sklearn import model_selection, datasets, svm
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, Aer, QuantumRegister, ClassicalRegister, execute

In [2]:
iris=datasets.load_iris()
X=iris.data[:100]
Y=iris.target[:100]
X_train, X_test, Y_train, Y_test=model_selection.train_test_split(X, Y, test_size=0.33, random_state=42)

In [3]:
N=X.shape[1]
print(N)


4


In [4]:
def initilialize_circuit(N):
    q=QuantumRegister(N)
    c=ClassicalRegister(1)
    qc=QuantumCircuit(q,c)
    return qc,c

In [5]:
def feature_map(qc, X):
    for i,x in enumerate(X):
        qc.rx(x, i)
    return qc

In [6]:
X[5]

array([5.4, 3.9, 1.7, 0.4])

In [7]:
def variational_circuit(qc, theta, N):
    for i in range(N-1):
        qc.cx(i, i+1)
    qc.cx(N-1, 0)
    for i in range(N):
        qc.ry(theta[i], i)
    return qc

In [8]:
def quantum_nn(X, theta, simulator=True):
    qc,c=initilialize_circuit(N)
    qc=feature_map(qc, X)
    qc.barrier()
    qc=variational_circuit(qc, theta, N)
    qc.barrier()
    qc.measure(0,c)
    qc.draw('mpl')


    shots=1E4
    backend=Aer.get_backend('qasm_simulator')
    jobs=execute(qc, backend, shots=shots)
    result=jobs.result()
    counts=result.get_counts(qc)

    return counts['1']/shots

In [9]:
def loss_fn(pred, target):
    return (pred-target)**2

In [10]:
import copy
def gradient(X, Y, theta):
    delta=0.01
    grad=[]

    for i in range(len(theta)):
        dtheta=copy.copy(theta)
        dtheta[i]+=delta

        pred1=quantum_nn(X, dtheta)
        pred2=quantum_nn(X, theta)

        grad.append((loss_fn(pred1, Y)-loss_fn(pred2, Y))/delta)

    return np.array(grad)

In [11]:
def accuracy(X, Y, theta):
    counter=0

    for X_i, Y_i in zip(X, Y):
        prediction=quantum_nn(X_i, theta)

        if prediction<0.5 and Y_i==0:
            counter+=1
        if prediction>=0.5 and Y_i==1:
            counter+=1

    return counter/len(Y)

In [ ]:
lr=0.05
loss_list=[]
epochs=12
theta=np.ones(N)

print('Epoch \t Loss \t Accuracy')

for i in range(epochs):
    loss_tmp=[]
    for X_i, Y_i in zip(X_train, Y_train):
        pred=quantum_nn(X_i, theta)
        loss_tmp.append(loss_fn(pred, Y_i))
        theta=theta-lr*gradient(X_i, Y_i, theta)

    loss_list.append(np.mean(loss_tmp))
    acc=accuracy(X_train, Y_train, theta)

    print(f'{i} \t {loss_list[-1]:.3f} \t {acc:.3f}')

Epoch 	 Loss 	 Accuracy


C:\Users\shrut\AppData\Local\Programs\Python\Python312\Lib\site-packages\qiskit\visualization\circuit\matplotlib.py:266: FutureWarning: The default matplotlib drawer scheme will be changed to "iqp" in a following release. To silence this warning, specify the current default explicitly as style="clifford", or the new default as style="iqp".
  self._style, def_font_ratio = load_style(self._style)


0 	 0.297 	 0.164
1 	 0.306 	 0.164
2 	 0.312 	 0.179
3 	 0.314 	 0.179
4 	 0.314 	 0.179
5 	 0.308 	 0.149
6 	 0.297 	 0.179


In [ ]:
plt.plot(loss_list)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
accuracy(X_test, Y_test, theta)